# Experiment

Experiments were run using WANDB as experiment management tool, using sweeps to search for hyperparameters.

### Setup

In [ ]:
%load_ext autoreload

In [ ]:
import sys

sys.path.append("..")

In [ ]:
%autoreload 2

from time import time as clock_timer

import numpy as np
import pandas as pd
import torch
import wandb
from matplotlib import pyplot as plt

from src.experiment.reporters import ReporterList, RepressionReporter, PredictionPlotReporter, TrainingHistoryPlotReporter
from src.datasets.data_loaders import TorchTimeSeriesCsvDataLoader
from src.experiment.runner import WandbRunner
from src.experiment.models import fmlp_from_parameters, time_series_mlp_from_parameters, frequency_only_mlp_from_parameters
from src.trainers import callbacks
from src.metrics.regression import regression_report, regression_score
from src.models.fmlp import TimeFrequencyLinear
from src.models.utils import count_trainable_parameters

### Reporters

Reporters are standalone classes used to report experiment results to WANBD dashboard, for particular benchmark. Two of the are implemented:
* `RepressionReporter` - reporting regression metrics, such as RMSE or $R^2$
* `PredictionPlotReporter` - plotting model predictions against system measurements along with error

In [ ]:
reporters = ReporterList([RepressionReporter(), PredictionPlotReporter()])

### Data Loader

Data loader is utility class used for training and testing the model (also supports valiation), which requires configuration for given benchmark.
Data loader provided in example is configured for Wiener-Hammerstein benchmark and model predicting one-step ahead.

In [ ]:
data_loader = TorchTimeSeriesCsvDataLoader(
        dataset_path=r"data/WienerHammerBenchmark.csv",
        input_columns=["uBenchMark"],
        output_columns=["yBenchMark"],
        test_size=88_000,
        batch_size=512,
        validation_size=0.1,
        window_generation_config=dict(
            shift=1,
            forward_input_window_size=256,
            forward_output_window_size=256,
            forward_output_mask=256 - 1),
        dtype=torch.float32,
    )


### Model Selection

Simple util for creating model instance using given parameters and short name for the model.

In [ ]:
def model_from_parameters(parameters):
    model_name_to_build_func = {
        "FP": frequency_only_mlp_from_parameters,
        "MLP": time_series_mlp_from_parameters,
        "FMLP": fmlp_from_parameters,
    }

    return model_name_to_build_func.get(parameters["model"])(parameters)

### Runner Config

Configuration for experiment runner, it supports number of parameters:
* `device` - torch device to run on, CUDA or CPU
* `callback_parameters` - parameters provided to Trainer, for more details see description of callbacks in `src/trainers/callbacks.py`
* `checkpoint_parameters` - parameters provided to Trainer, for more details see description of callbacks in `src/trainers/checkpoints.py`

In [ ]:
runner_config = {
    "callback_parameters": {
        "names": ["early_stopping", "regression_report", "training_timeout"],
        "parameters": [
            {"metric_name": "RMSE", "patience": 1000, "moving_average_window_size": 100, "delta": 1},  # early stopping
            {"run_period": 1, "metric_names": ["MSE", "RMSE", "R2"]},  # reporter
            {"max_training_time": 24 * 60 * 60}  # timeout
        ],
    },
    "checkpoint_parameters": {
        "names": ["simple_model_checkpoint", "best_model_checkpoint"],
        "parameters": [
            {"model_save_path": "models/test", "run_period": 100},  # simple_model_checkpoint
            {"model_save_path": "models/test", "metric_name": "RMSE"}  # best_model_checkpoint
        ],
        "restore_from": "BestModelCheckpoint",
    },
    "device": "cuda",
}

runner = WandbRunner(
    runner_config,
    model_from_parameters=model_from_parameters,
    data_loader=data_loader,
    reporters=reporters,
)

### Sweep Config

Sweep config is a `dict` providing parameters to WANDB sweep and parameters used to create model instance along training parameters, such as optimizer or number of epochs.

In [ ]:
sweep_config = {
    "name" : "sweep-name",
    "method" : "grid",  # supports grid, random or bayesian search
    "metric": {
        "name": "NRMSE",
        "goal": "minimize",
    },
    # parameters provided to model and trainig script
    "parameters" : {
        "optimizer": {
            "values": ["adam"]
        },
        "n_epochs": {
            "values": [100_000],
        },
        "loss_function":{
            "values": ["mse"]
        },
        "n_input_time_steps": {
            "values": [256],  # this needs to be aligned with data_loader configuration
            },
        "n_output_time_steps": {
            "values": [1]
        },
        "n_input_state_variables": {
             "values": [1],
        },
        "n_output_state_variables": {
            "values": [1],
        },
        # model parameters
        "n_hidden_layers": {
            "values": [5],
        },
        "activation": {
            "values": ["gelu"]
        },
        "n_hidden_time_steps": {
            "values": [64],
        },
        "n_hidden_state_variables": {
            "values": [1],
        },
        "model": {
            "values": ["MLP", "FMLP", "FP"],
        }
    }
}

This will run 3 training experiments with the same parameters for each model, `MLP`, `FMLP` and `FP`.

In [ ]:
runner.run_sweep(sweep_config, project_name="fmlp", n_runs=3)